In [6]:
import pandas as pd
import numpy as np

# Load dataset
file_path = "alldata_fi.csv"
df = pd.read_csv(file_path)

# Convert Date column to datetime and sort
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Select relevant columns
columns_needed = ['option_name', 'Closing_Price_(Option)', 'Exercise_Price',
                  'Spot_Price_of_Asset', 'riskfreerate', 'time_to_expiration']
df = df[columns_needed]
summary_stats = df.describe()
summary_stats
# Save to Excel
summary_stats.to_excel("summary_statistics.xlsx")

In [9]:
# Load dataset
file_path = "alldata_fi.csv"
df = pd.read_csv(file_path)

# Convert Date column to datetime and sort
df['Date'] = pd.to_datetime(df['Date'])

# Select relevant columns
columns_needed = ['Date', 'option_name', 'Closing_Price_(Option)', 'Exercise_Price',
                  'Spot_Price_of_Asset', 'riskfreerate', 'time_to_expiration']
df = df[columns_needed]

df = df.sort_values(by=['option_name', 'Date'])
df.set_index('Date', inplace=True)

#Calculate daily log returns for each option_name group
df['log_return'] = df.groupby('option_name')['Spot_Price_of_Asset'].transform(lambda x: np.log(x / x.shift(1)))

#Calculate 30-day rolling historical volatility (annualized with √240)
def compute_vol(group):
    group['volatility'] = group['log_return'].rolling(window=30).std() * np.sqrt(240)
    return group

df = df.groupby('option_name', group_keys=False).apply(compute_vol)

# Drop log_return
df.drop(columns=['log_return'], inplace=True)
df = df.dropna(subset=['volatility'])

summary_stats = df.describe()
summary_stats
# Save to Excel
summary_stats.to_excel("summary_statistics+volatility.xlsx")

<ipython-input-9-a91bff7755c9>:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('option_name', group_keys=False).apply(compute_vol)
